<a href="https://colab.research.google.com/github/SDS-AAU/SDS-2021/blob/master/SDS-2021/static/workshops/2021/M2W2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install tweet-preprocessor -qq

In [1]:
import pandas as pd
import preprocessor as prepro


In [2]:
data_congress = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pol_tweets.gz')

In [3]:
data_congress

,text,labels
340675,RT @GreenBeretFound Today we remember Sgt. 1st...,0
289492,"Yes, yes, yes, yes and yes. 😷 #JerseyStrong 💪🏾...",1
371088,Made new friends this afternoon delivering mas...,1
82212,RT @TXMilitary Happening TODAY: Pilots with th...,0
476047,RT @SteveScalise President Trump's legal team ...,0
...,...,...
61499,Outrageous.\n\nBrave health care workers are p...,0
185562,RT @dskolnick .@RepTimRyan proposes up to $3K ...,1
354040,It is clear that the #HeroesAct will help tens...,1
708686,Democrats are talking about Bolton and Mulvane...,0


In [17]:
prepro.set_options(prepro.OPT.URL, prepro.OPT.EMOJI, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

In [18]:
data_congress['text_clean'] = data_congress['text'].map(lambda t: prepro.clean(t))

In [20]:
data_congress['text_clean'] = data_congress['text_clean'].str.replace('#','')

In [ ]:
import spacy
nlp = spacy.load("en")

In [ ]:
nlp_text = nlp('roman is confusing and we kind of need a brake right now.')

In [ ]:
[token.text for token in nlp_text if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop]

['roman', 'confusing', 'kind', 'brake', 'right']

In [ ]:
tokens = []

for tweet in nlp.pipe(data_congress.sample(1000)['text_clean']):
  tweet_tok = [token.lemma_.lower() for token in tweet if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] and not token.is_stop] 
  tokens.extend(tweet_tok)

In [ ]:
bootstrap_dictionary = list(set(tokens))

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary=bootstrap_dictionary)
X = vectorizer.fit_transform(data_congress['text_clean'])

In [53]:
X.shape

(50000, 4092)

In [62]:
# requires instantiated vectorizer e.g. TFIDF

def preprocessTweets(data_tweets):
  clean_text = data_tweets.map(lambda t: prepro.clean(t))
  clean_text = clean_text.str.replace('#','')
  return vectorizer.transform(clean_text)

In [54]:
y = data_congress['labels']

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21)

In [56]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(multi_class="ovr") # since we have 3 classes
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
model.score(X_test, y_test)

0.7538

In [5]:
data_tweets = pd.read_json('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/pres_debate_2020.gz')

In [6]:
data_tweets

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,reply_to,translate,trans_src,trans_dest
0,1318944772183281664,1318944772183281664,2020-10-21 15:58:33,2020-10-21 15:58:33,0,,Still time to register: Students can join the ...,en,[presidentialdebate2020],[],1130857348921036802,1130857348921036800,UVADemocracy,UVA Democracy Initiative,3,15,https://twitter.com/UVADemocracy/status/131894...,[https://bit.ly/349NTIU],[https://pbs.twimg.com/media/Ek3UXC1X0AAw47D.png],1,https://pbs.twimg.com/media/Ek3UXC1X0AAw47D.png,2,0,2,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
1,1318938583122743296,1318938583122743296,2020-10-21 15:33:57,2020-10-21 15:33:57,0,,Be prepared for Trump to railroad Thursday’s d...,en,[presidentialdebate2020],[],243363569,243363569,kevinjguest,Kevin Guest,3,15,https://twitter.com/kevinjguest/status/1318938...,[],[],0,,0,0,0,https://twitter.com/donaldjtrumpjr/status/1318...,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
2,1318932554897031168,1318932554897031168,2020-10-21 15:10:00,2020-10-21 15:10:00,0,,Join us tomorrow from 5-8pm as @michaelpleahy ...,en,[presidentialdebate2020],[],26819436,26819436,TalkradioWLAC,Talkradio WLAC,3,15,https://twitter.com/TalkradioWLAC/status/13189...,[https://wlac.iheart.com/calendar/event/5f8df3...,[],0,,0,0,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
3,1318928783169245184,1318928783169245184,2020-10-21 14:55:01,2020-10-21 14:55:01,0,,Wanna bet #ProudBoys comes up #PresidentialDeb...,en,"[proudboys, presidentialdebate2020]",[],298018860,298018860,PBPoliticsFins,Antonio Fins,3,14,https://twitter.com/PBPoliticsFins/status/1318...,[https://www.palmbeachpost.com/story/news/2020...,[],0,,0,0,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
4,1318927150247018496,1318927150247018496,2020-10-21 14:48:31,2020-10-21 14:48:31,0,,RT College Tour @BelmontUniv was spotless. Gor...,en,"[musiccity, presidentialdebate2020]",[],4159192877,4159192877,12thSouth,12th South,3,14,https://twitter.com/12thSouth/status/131892715...,[],[https://pbs.twimg.com/media/Ek3CBOhXYAIpo8C.j...,1,https://pbs.twimg.com/media/Ek3CBOhXYAIpo8C.jpg,0,0,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8806,1311705601597222912,1311705601597222912,2020-10-01 16:32:40,2020-10-01 16:32:40,0,,Check out my Gig on Fiverr: do email marketing...,en,"[october1st, worsttrumpever, thursdaythoughts,...",[],1294253523769667586,1294253523769667584,kvngmujebo,Kvngmujebo,4,16,https://twitter.com/kvngmujebo/status/13117056...,[https://www.fiverr.com/share/1KRDaK],[],0,,0,0,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
8807,1311705488531156992,1311705488531156992,2020-10-01 16:32:13,2020-10-01 16:32:13,0,,This was made in 2012! It’s exactly like what ...,en,"[democrats, republicans, presidentialdebate202...",[],25635056,25635056,AZLatina480,Jessica Williams,4,16,https://twitter.com/AZLatina480/status/1311705...,[],[],0,,0,0,0,https://twitter.com/JonnyEthco/status/13113395...,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
8808,1311705196657958912,1311705196657958912,2020-10-01 16:31:03,2020-10-01 16:31:03,0,,How you finna lose two swing states with one q...,en,[presidentialdebate2020],[],382541164,382541164,SampsonRaySimon,Sampson Ray Simon,4,16,https://twitter.com/SampsonRaySimon/status/131...,[],[],0,,0,0,0,,PresidentialDebate2020,,,,"{'user_id': None, 'username': None}",,,
8809,1311704929090891776,1311704929090891776,2020-10-01 16:30:00,2020-10-01 16:30:00,0,,"This morning on the @ArleneBynonShow, @gmacofg...",en,"[erinotoole, blanchet, houseofcommons, trudeau...",[],1343036322,1343036322,SXMC

In [65]:
X_new_tweets = preprocessTweets(data_tweets['tweet'])

In [68]:
predictions_new_tweets = model.predict_proba(X_new_tweets)

In [69]:
predictions_new_tweets

array([[0.18576599, 0.81423401],
       [0.362734  , 0.637266  ],
       [0.2135539 , 0.7864461 ],
       ...,
       [0.3949195 , 0.6050805 ],
       [0.36839689, 0.63160311],
       [0.71442461, 0.28557539]])

In [73]:
data_tweets['dem_probability'] = predictions_new_tweets[:,1]

In [77]:
data_tweets.sort_values('dem_probability')['tweet'].iloc[-1]

'"President Trump has Jewish grandchildren" is the new "I have black friends". #PresidentialDebate2020'